In [33]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *
from fastai import sgdr

In [2]:
PATH = '/home/ubuntu/projects/dantegen/'

In [3]:
#fpath = get_file(PATH + 'data/raw/divcomm.txt', origin="http://www.gutenberg.org/files/1012/1012-0.txt")
fpath = PATH + 'data/raw/divcomm.txt'
text = open(fpath,encoding='utf8').read().lower()
text = text[932:-19658]

In [4]:
print(text[:18])

la divina commedia


In [5]:
chars = sorted(list(set(text)))
chars.insert(0, "\0")
vocab_size = len(chars)
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}
idx = [char_indices[c] for c in text]

In [6]:
print(vocab_size)
"".join(chars)

56


'\x00\n !(),-.:;?abcdefghijlmnopqrstuvxyz«»àäèéëìïòóöùü—‘’“”•'

In [7]:
maxlen = 40
sentences = []
next_chars = []
c_in_dat = [[idx[i+j] for i in range(maxlen)] for j in range(len(idx)-maxlen-1)]
xs = np.stack(c_in_dat, axis=0)
c_out_dat = [idx[j+maxlen] for j in range(len(idx)-maxlen-1)]
y = np.stack(c_out_dat)

In [8]:
xs.shape,y.shape

((561053, 40), (561053,))

In [9]:
n_hidden = 256
n_fac = 24

In [10]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]))#, dim=-1)

In [11]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [12]:
val_idx = get_cv_idxs(len(idx)-maxlen-1)
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [13]:
it = iter(md.trn_dl)
*xs,yt = next(it)

In [14]:
t = m.e(V(torch.stack(xs)))
t.size()

torch.Size([40, 512, 24])

In [15]:
ht = V(torch.zeros(1, 512,n_hidden))
outp, hn = m.rnn(t, ht)
outp.size(), hn.size()

(torch.Size([40, 512, 256]), torch.Size([1, 512, 256]))

In [16]:
t = m(*V(xs)); t.size()

torch.Size([512, 56])

In [17]:
fit(m, md, 4, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.80164  1.79242]                                
[ 1.       1.66657  1.6531 ]                                
[ 2.       1.60474  1.61099]                                
[ 3.       1.60834  1.62478]                                



In [18]:
set_lrs(opt, 1e-4)

In [19]:
fit(m, md, 2, opt, F.nll_loss)

A Jupyter Widget

[ 0.       1.56124  1.59216]                                
[ 1.       1.55215  1.5851 ]                                



In [26]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [27]:
def get_next_n(inp = "nel mezzo del cammin di nostra vita\n  mi", n = 500):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [28]:
print(get_next_n())

nel mezzo del cammin di nostra vita
  mi fare a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si rispuose a la sua che si ris


In [34]:
PATH='/home/ubuntu/projects/dantegen/data/raw/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

divcomm.txt  trn/  val/


In [35]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=40; n_fac=24; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

RuntimeError: given sequence has an invalid size of dimension 2: 0